In [16]:
# GOAL DESCRIPTION
# COMPLETION TIME AND OTHER STUFF
# GOAL STRUCTURE
# GOAL AFFECT VS CONFLICT
# MOTIVATION RELATION TO STUFF
# AFFECT RELATION TO STUFF
# 2 APPLICATION
# 1 APPLICATION
# CLEANING
# POSITIVE AFFET SHOULD BE POSITIVELY RELATED TO GOAL PROGRESS WHEN CONFLICTING GOALSA ARE NOT PRESENT


In [17]:
import pandas as pd
import numpy as np
import config
import os
import openai
import time

import json

import random
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [18]:
project_dir = "/Users/farhan/Desktop/Courses/goal_research/"
daily_goal_embedding_path = os.path.join(project_dir, "data/labelled_goals/daily_goal_embeddings.csv")
cols = [
    "ParticipantIdentifier", "ResultIdentifier", 
    "Answers", "trial_date",  "label", "embedding"
    ]


In [19]:
DF = pd.read_csv(daily_goal_embedding_path)[cols]
DF = DF.pivot_table(
    index=['ParticipantIdentifier', 'trial_date'],  # Correct the spelling of 'ParticipantIdentifier'
    columns='ResultIdentifier',                      # Correct the spelling of 'resultsIdentifier'
    values=['Answers', 'label', 'embedding'],
    aggfunc='first'
)


In [20]:
DF = DF.dropna().reset_index()
DF.columns = [f'{col[0]}_{col[1]}' if isinstance(col, tuple) else col for col in DF.columns]
DF.columns = [col if 'labels' not in col and 'embeddings' not in col else f"{col[1]}" for col in DF.columns]
DF.columns = DF.columns.str.rstrip('_')
DF.columns

Index(['ParticipantIdentifier', 'trial_date', 'Answers_DAILY_goal1_set',
       'Answers_DAILY_goal2_set', 'embedding_DAILY_goal1_set',
       'embedding_DAILY_goal2_set', 'label_DAILY_goal1_set',
       'label_DAILY_goal2_set'],
      dtype='object')

In [21]:
categories = list(set(DF["label_DAILY_goal1_set"].to_list() + DF["label_DAILY_goal2_set"].to_list()))
participant_list = list(set(DF["ParticipantIdentifier"].to_list()))

In [22]:
self_rep = pd.read_csv("/Users/farhan/Desktop/Courses/goal_research/data/self_report_measures.csv")
DF = self_rep.merge(DF, how="inner", on=["ParticipantIdentifier", "trial_date"])
DF['continuation'] = False
DF['original_goal_date'] = pd.NaT

In [23]:
DF['trial_date'] = pd.to_datetime(DF['trial_date'])

DF['DAILY_goal1_report'].fillna(0, inplace=True)  # Replace NaN with 0
DF['DAILY_goal2_report'].fillna(0, inplace=True)  # Replace NaN with 0


/var/folders/57/9mrlx9kx6xsfb5tsnbtdb9d40000gn/T/ipykernel_93582/3071481819.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  DF['DAILY_goal1_report'].fillna(0, inplace=True)  # Replace NaN with 0
/var/folders/57/9mrlx9kx6xsfb5tsnbtdb9d40000gn/T/ipykernel_93582/3071481819.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we a

In [24]:
DF

,Unnamed: 0,ParticipantIdentifier,trial_date,DAILY_goal1_confidence,DAILY_goal1_consequences,DAILY_goal1_effort,DAILY_goal1_importance,DAILY_goal2_confidence,DAILY_goal2_consequences,DAILY_goal2_effort,...,DAILY_goal2_report,DAILY_goal2_interaction_eachOther,Answers_DAILY_goal1_set,Answers_DAILY_goal2_set,embedding_DAILY_goal1_set,embedding_DAILY_goal2_set,label_DAILY_goal1_set,label_DAILY_goal2_set,continuation,original_goal_date
0,0,0501ba67-3406-4779-aff1-878a0e9f7885,2022-09-30,4.0,3.0,6.0,5.0,6.0,1.0,7.0,...,0.0,NaN,"Study history and psychology, practice the violin",Do exercise,"[-0.038126230239868164, -0.021308906376361847,...","[-0.012392040342092514, 0.017055129632353783, ...",School,Exercise,False,NaT
1,3,0501ba67-3406-4779-aff1-878a0e9f7885,2022-10-03,5.0,7.0,7.0,7.0,7.0,3.0,7.0,...,0.0,NaN,review,movie review,"[0.03161436691880226, 0.029465556144714355, -0...","[-0.02972441539168358, 0.03863658756017685, -0...",School,Culture,False,NaT
2,7,0501ba67-3406-4779-aff1-878a0e9f7885,2022-10-07,4.0,3.0,5.0,4.0,6.0,1.0,5.0,...,30.0,NaN,practice the violin,Work out,"[-0.01599280908703804, -0.0059240879490971565,...","[-0.03653150796890259, 0.02275843359529972, -0...",Hobby,Exercise,False,NaT
3,9,0501ba67-3406-4779-aff1-878a0e9f7885,2022-10-09,6.0,1.0,3.0,3.0,5.0,1.0,4.0,...,100.0,NaN,Exercise,Keep diet,"[-0.013323132880032063, -0.011583465151488781,...","[0.011315252631902695, -0.01544771995395422, -...",Exercise,Diet,False,NaT
4,13,0501ba67-3406-4779-aff1-878a0e9f7885,2022-10-13,3.0,5.0,5.0,7.0,5.0,4.0,3.0,...,0.0,NaN,study history,finish readings for psy,"[0.009109092876315117, -0.0389975905418396, 0....","[0.029414815828204155, 0.034327395260334015, 0...",School,School,False,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,4385,fed3a16b-c5a3-4a84-a111-32b7574e04b5,2022-12-07,5.0,5.0,5.0,3.0,4.0,5.0,4.0,...,64.0,0.0,stud,study,"[-0.012145000509917736, -0.020833661779761314,...","[0.01715385541319847, -0.015046305023133755, 0...",School,School,False,NaT
1241,4387,fed3a16b-c5a3-4a84-a111-32b7574e04b5,2022-12-09,5.0,5.0,5.0,4.0,4.0,5.0,3.0,...,69.0,0.0,bio,calc,"[0.036754854023456573, 0.006017761304974556, -...","[0.006103042047470808, -0.005450013093650341, ...",School,School,False,NaT
1242,4391,fed3a16b-c5a3-4a84-a111-32b7574e04b5,2022-12-15,7.0,7.0,7.0,6.0,6.0,6.0,6.0,...,91.0,0.0,fun,snow,"[-0.004395645577460527, 0.015698228031396866, ...","[-0.017143726348876953, -0.007770814001560211,...",Other,Other,False,NaT
1243,4392,fed3a16b-c5a3-4a84-a111-32b7574e04b5,2022-12-16,6.0,6.0,6.0,7.0,6.0,6.0,6.0,...,82.0,0.0,secret Santa,fun,"[-0.007166695315390825, -0.023291759192943573,...","[-0.004395645577460527, 0.015698228031396866, ...",Friends,Hobby,False,NaT


In [25]:
x = pd.DataFrame
y = False
for participant in participant_list:
    tempDF = DF.loc[DF["ParticipantIdentifier"] == participant]

    for idx, goal in tempDF.iterrows():
        if tempDF.at[idx, 'continuation'] == True:
             continue
        tempDF.at[idx, 'original_goal_date'] =  goal['trial_date']
        # Check next 7 days
        for idx2, next_goal in tempDF.iterrows():
            if 0 < (next_goal['trial_date'] - goal['trial_date']).days <= 7:
                # if next_goal['label_DAILY_goal1_set'] == goal['label_DAILY_goal1_set']:
                    # Calculate cosine similarity
                    goal_embedding = json.loads(goal['embedding_DAILY_goal1_set'])
                    next_goal_embedding = json.loads(next_goal['embedding_DAILY_goal1_set'])
                    similarity = cosine_similarity([goal_embedding], [next_goal_embedding])
                    
                    # Check thresholds
                    curr_goal_completion = int(next_goal['DAILY_goal1_report']);
                    next_goal_completion = int(goal['DAILY_goal1_report']);
                    completion_diff = abs(next_goal_completion - curr_goal_completion)

                    if similarity > 0.75 and completion_diff <= 15:
                        tempDF.at[idx2, 'continuation'] = True
                        tempDF.at[idx2, 'original_goal_date'] = goal['trial_date']  # Store original goal date
                        break  # Exit loop once a continuation is found
    
    if y == False:
         y = True
         x = tempDF
    else:
         x = pd.concat([x, tempDF])

In [26]:
x

,Unnamed: 0,ParticipantIdentifier,trial_date,DAILY_goal1_confidence,DAILY_goal1_consequences,DAILY_goal1_effort,DAILY_goal1_importance,DAILY_goal2_confidence,DAILY_goal2_consequences,DAILY_goal2_effort,...,DAILY_goal2_report,DAILY_goal2_interaction_eachOther,Answers_DAILY_goal1_set,Answers_DAILY_goal2_set,embedding_DAILY_goal1_set,embedding_DAILY_goal2_set,label_DAILY_goal1_set,label_DAILY_goal2_set,continuation,original_goal_date
987,3445,c2097f36-4ca3-4537-856d-a649d1557553,2022-10-09,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,35.0,0.0,relax,relax again,"[0.0666937306523323, -0.004000754561275244, -0...","[0.06920930743217468, 0.0006226471741683781, -...",MentalPersonal_Health,MentalPersonal_Health,False,2022-10-09
988,3446,c2097f36-4ca3-4537-856d-a649d1557553,2022-10-10,6.0,6.0,6.0,6.0,4.0,4.0,4.0,...,96.0,0.0,do chem peer scholar,study for bio midterm,"[-0.0014588343910872936, -0.04640701785683632,...","[0.0011399213690310717, 0.0021749420557171106,...",School,School,False,2022-10-10
989,3449,c2097f36-4ca3-4537-856d-a649d1557553,2022-10-13,6.0,6.0,4.0,6.0,6.0,5.0,4.0,...,86.0,0.0,study for bio midterm,work on business essay,"[0.0011399213690310717, 0.0021749420557171106,...","[-0.005441735498607159, 0.019361430779099464, ...",School,School,False,2022-10-13
990,3450,c2097f36-4ca3-4537-856d-a649d1557553,2022-10-14,7.0,7.0,3.0,6.0,4.0,4.0,4.0,...,92.0,0.0,study for bio midterm,complete data section of FLR,"[0.0011399213690310717, 0.0021749420557171106,...","[0.016017740592360497, 0.043732840567827225, 0...",School,School,True,2022-10-13
991,3452,c2097f36-4ca3-4537-856d-a649d1557553,2022-10-16,6.0,5.0,1.0,5.0,5.0,3.0,3.0,...,17.0,0.0,complete bio lab 4 assignment,work on business stuff,"[0.005013881716877222, 0.015968315303325653, 0...","[-0.010066596791148186, 0.03734434396028519, -...",School,Work/Job,False,2022-10-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,666,25ca39d7-4279-48fd-903f-d0927adadb77,2022-10-09,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,59.0,0.0,Workout,Study,"[-0.029043760150671005, 0.004490229766815901, ...","[0.008811529725790024, -0.010646408423781395, ...",Exercise,School,True,2022-10-08
210,668,25ca39d7-4279-48fd-903f-d0927adadb77,2022-10-11,5.0,5.0,5.0,6.0,5.0,5.0,5.0,...,72.0,0.0,Study,Workout,"[0.008811529725790024, -0.010646408423781395, ...","[-0.029043760150671005, 0.004490229766815901, ...",School,Exercise,False,2022-10-11
211,669,25ca39d7-4279-48fd-903f-d0927adadb77,2022-10-13,4.0,2.0,4.0,6.0,4.0,4.0,4.0,...,75.0,0.0,Workout,Study,"[-0.029043760150671005, 0.004490229766815901, ...","[0.008811529725790024, -0.010646408423781395, ...",Exercise,School,False,2022-10-13
212,671,25ca39d7-4279-48fd-903f-d0927adadb77,2022-10-15,5.0,5.0,5.0,6.0,5.0,5.0,5.0,...,66.0,0.0,Workout,Study,"[-0.029043760150671005, 0.004490229766815901, ...","[0.00878792256116867, -0.010670071467757225, 0...",Exercise,School,True,2022-10-13


In [27]:
x["continuation"].value_counts()

continuation
False    1135
True      110
Name: count, dtype: int64

## GROUP BY GOAL CATEGORY
## FOR EACH GOAL, CALCULATE SIMILARITY WITH NEXT 7 DAYS, 
## GOAL CONSIDERED SAME AND ON GOING IF SIMILARITY GREATER THAN 85 %, LESS THAN 7 DAYS GAP, COMPLETION GWITHIN 10% OF PREVIOUS DAY
## IF SO, MARK THIS GOAL AS CONTINUAITION
## MOVE ON TO NEXT ITERATION AND CONTINUE ON TO NEXT GOAL ONLY IF IT IS NOT A CONTINUATION.
## COMPLETE FOR ALL

In [28]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Example DataFrame
data = {
    'goal_id': [1, 2, 3, 4, 5, 6],
    'label': ['A', 'A', 'B', 'B', 'A', 'B'],
    'completion_percentage': [50, 60, 55, 30, 70, 80],
    'trial_date': pd.to_datetime(['2024-09-01', '2024-09-03', '2024-09-05', '2024-09-10', '2024-09-12', '2024-09-15']),
    'embedding': [np.random.rand(10) for _ in range(6)]  # Replace with your actual embeddings
}

df = pd.DataFrame(data)

# Function to determine continuation
def mark_continuations(df):
    # Initialize columns for continuations and original goal date
    df['continuation'] = False
    df['original_goal_date'] = pd.NaT

    for idx, goal in df.iterrows():
        # Filter for goals in the same category
        same_category = df[(df['label'] == goal['label']) & (df['trial_date'] > goal['trial_date'])]
        
        # Check next 7 days
        for _, next_goal in same_category.iterrows():
            if (next_goal['trial_date'] - goal['trial_date']).days <= 7:
                # Calculate cosine similarity
                sim = cosine_similarity([goal['embedding']], [next_goal['embedding']])[0][0]
                
                # Check thresholds
                completion_diff = abs(next_goal['completion_percentage'] - goal['completion_percentage'])
                if sim > 0.85 and completion_diff <= 10:
                    df.at[idx, 'continuation'] = True
                    df.at[idx, 'original_goal_date'] = goal['trial_date']  # Store original goal date
                    break  # Exit loop once a continuation is found

    return df

# Apply the function
df = mark_continuations(df)

# Display the resulting DataFrame
print(df)


   goal_id label  completion_percentage trial_date  \
0        1     A                     50 2024-09-01   
1        2     A                     60 2024-09-03   
2        3     B                     55 2024-09-05   
3        4     B                     30 2024-09-10   
4        5     A                     70 2024-09-12   
5        6     B                     80 2024-09-15   

                                           embedding  continuation  \
0  [0.9885463062303025, 0.5703854844778821, 0.771...         False   
1  [0.3519798474051964, 0.9244784829960944, 0.010...         False   
2  [0.45525991526227816, 0.16303611982519517, 0.5...         False   
3  [0.17152787477389508, 0.5381210353663299, 0.91...         False   
4  [0.7263088506598849, 0.6772616577382743, 0.011...         False   
5  [0.40675387490852055, 0.9240809285117465, 0.79...         False   

  original_goal_date  
0                NaT  
1                NaT  
2                NaT  
3                NaT  
4              